In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.mixture import BayesianGaussianMixture
import matplotlib.pyplot as plt
import matplotlib
from sklearn.mixture import GaussianMixture as standard_GaussianMixture
from scipy.stats import multivariate_normal
import argparse
from sklearn.manifold import TSNE
# import torch
from sklearn.neural_network import MLPClassifier
import itertools
import seaborn as sns
import zipfile

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from econml.dml import CausalForestDML

from econml.iv.dml import DMLIV
from sklearn.linear_model import Lasso,LassoCV

In [4]:
data=pd.read_csv("all_patients_data.csv")
data=data.drop(columns=["icustay_id","hadm_id","intime"	,"outtime",	"dbsource",	"suspected_infection_time_poe",
"antibiotic_time_poe","blood_culture_time","gender","ethnicity","icu_los","hosp_los","specimen_poe","first_service",'subject_id'
])


# data['colloid_bolus']=data["colloid_bolus"].fillna(0)
data=data.fillna(data.mean())
# infection_specimen=OneHotEncoder().fit_transform(data["first_service"].to_numpy().reshape(-1,1))

train_data,test_data=train_test_split(data,test_size=0.2,random_state=505)

train_label=train_data["thirtyday_expire_flag"]
train_data=train_data.drop(columns=["thirtyday_expire_flag","hospital_expire_flag"])

test_label=test_data["thirtyday_expire_flag"]
test_data=test_data.drop(columns=["thirtyday_expire_flag","hospital_expire_flag"])

KeyError: "['icustay_id' 'hadm_id' 'intime' 'outtime' 'dbsource'\n 'suspected_infection_time_poe' 'antibiotic_time_poe' 'blood_culture_time'\n 'gender' 'ethnicity' 'icu_los' 'hosp_los' 'specimen_poe' 'first_service'\n 'subject_id'] not found in axis"

# Causal Forest

In [10]:
%%capture

np.random.seed(505)
causal_forest_estimator = CausalForestDML(discrete_treatment=True,discrete_outcome=True,n_estimators =100,cv=3,max_depth=10
                    )
causal_forest_estimator.fit(train_label,train_data['vent'],
                            X=train_data.drop(['vent'],axis=1),inference='auto')

In [11]:
HTE_estimate=causal_forest_estimator.effect(X=test_data.drop(['vent'],axis=1), 
                                T0=np.zeros(test_data.shape[0]),
                                T1=np.ones(test_data.shape[0]))

# XLearner

In [16]:
from econml.metalearners import XLearner
from sklearn.ensemble import GradientBoostingRegressor


In [19]:
%%capture

np.random.seed(505)
XLearner_model=XLearner(models=GradientBoostingRegressor(max_depth=5,random_state=505))

XLearner_model.fit(Y=train_label, T=train_data['vent'],
                    X=train_data.drop(['vent'],axis=1))                    

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


# DRLearner

In [13]:
from econml.dr import DRLearner
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [15]:
%%capture

DRLearner_model = DRLearner(model_propensity=RandomForestClassifier(n_estimators=100, min_samples_leaf=10),
                model_regression='auto',
                model_final=LassoCV(cv=3),random_state=505,discrete_outcome=True,
                featurizer=None)
DRLearner_model.fit(Y=train_label.ravel(), T=train_data['vent'],
                    X=train_data.drop(['vent'],axis=1))


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations 